In [ ]:
data = [(1, 1, "uno-uno"), 
        (1, 2, "uno-dos"), 
        (2, 3, "dos-tres"),
        (2, 7, "dos-siete"),
        (2, 10, "dos-diez"),
        (3, 11, "tres-once")]

df = spark.createDataFrame(data, ["uid", "value", "desc"])
df.toPandas()

In [ ]:
df.createOrReplaceTempView("users")

df1 = spark.sql("SELECT * FROM "
                "(SELECT uid, value, desc, rank() OVER (PARTITION BY uid ORDER BY value DESC) AS rank FROM users) t "
                " WHERE rank = 1")
df1.show()

In [ ]:
from pyspark.sql.functions import col, row_number, rank
from pyspark.sql.window import Window

w = Window().partitionBy("uid").orderBy(col("value").desc())
# usar row_number para evitar duplicados en caso de empate
df2 = df.withColumn("rank", row_number().over(w)).where(col("rank") == 1)
df2.show()